In [1]:
import os
import pandas as pd
import requests
from tqdm import tqdm_notebook
import glob

In [2]:
os.listdir('../')

['.DS_Store',
 'requirements.txt',
 'synonyms',
 'yappy_search',
 '.gitignore',
 '.git',
 'notebooks',
 '.ipynb_checkpoints']

In [3]:
# Считываем данные из таблицы (CSV или Excel, в зависимости от формата)
file_path = '../../yappy_hackaton_2024_400k.csv'  # укажите правильный путь к файлу
df = pd.read_csv(file_path)  # если у вас Excel, используйте pd.read_excel(file_path)

In [4]:
df

,link,description
0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m..."
1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc...,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #rob...
2,https://cdn-st.rutubelist.ru/media/e9/e0/b47a9...,"#boobs , #красивыедевушки , #ass"
3,https://cdn-st.rutubelist.ru/media/87/43/b11df...,NaN
4,https://cdn-st.rutubelist.ru/media/d1/e7/642dc...,NaN
...,...,...
399995,https://cdn-st.rutubelist.ru/media/dd/6a/ac296...,#ceramics #handmade #ceramic #clay #ceramicscu...
399996,https://cdn-st.rutubelist.ru/media/aa/d3/4b4c3...,#красивыедевушки #красотки #грудь #boobs #секс...
399997,https://cdn-st.rutubelist.ru/media/73/05/3f80f...,"#лайфхаки , #эксперименты , #roblox , #игрушки..."
399998,https://cdn-st.rutubelist.ru/media/f8/2b/320f7...,#образ #lookbook #показ #неделямоды #капсула #...


In [5]:
import sys

sys.path.insert(
    0,
    "/home/jovyan/anvlasov/hack_yappy/git/",
)

In [6]:
dir_w_video:str = '../../data_video/'

In [7]:
# Получение списка всех файлов с расширением .mp4 в директории
video_files = glob.glob(os.path.join(dir_w_video, '*.mp4'))
# Сортировка списка файлов по первому значению при split('_')
sorted_video_files = sorted(video_files, key=lambda x: int(os.path.basename(x).split('_')[0]))

# Вывод отсортированного списка файлов
for video_file in sorted_video_files[:5]:
    print(video_file)

../../data_video/0_b0_e9_ef285e0241139fc611318ed33071_fhd.mp4
../../data_video/1_39_6c_b31bc6864bef9d8a96814f1822ca_fhd.mp4
../../data_video/2_e9_e0_b47a9df14a5e97942715e5e705c0_fhd.mp4
../../data_video/3_87_43_b11df3f344d0af773aac81e410ee_fhd.mp4
../../data_video/4_d1_e7_642dc2194fcdb69664f832d5f2dd_fhd.mp4


### Эксперимент по подбору модели whisper и запуску на batch

смотрим на 100 примерах

In [8]:
from yappy_search.utils import  extract_audio

In [9]:
video_path = sorted_video_files[0]

In [10]:
audio_output_dir:str= os.path.join(video_path.rsplit('/',2)[0],'data_audio')

In [11]:
audio_output_dir

'../../data_audio'

In [12]:
audio_paths = []
for video_path in tqdm_notebook(sorted_video_files[:100]):
    audio_file_path = os.path.join(
        audio_output_dir, os.path.basename(video_path).replace(".mp4", ".mp3")
    )
    if os.path.isfile(audio_file_path):
        pass
    else:
        audio_file_path = extract_audio(video_path, audio_output_dir)
    audio_paths.append(audio_file_path)

  0%|          | 0/100 [00:00<?, ?it/s]

### напрямую попробуем ускорить через модель

In [36]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

In [37]:
import time 

In [15]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model = model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [72]:
limit_song = 100
batch_size=32
language= 'ru'

In [75]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=batch_size,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device
)

In [76]:
transcriptions = []
start_time = time.time()
for audio_path in tqdm_notebook(audio_paths[:limit_song]):
    # audio_path = model_all.audio_transcription.extract_audio(video_path, audio_output_dir)
    transcription = pipe(audio_path)

    transcriptions.append(transcription)
time_work = time.time() - start_time

  0%|          | 0/100 [00:00<?, ?it/s]

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


KeyboardInterrupt: 

In [77]:
print(f"{time_work/limit_song} время работы на 1 записи в среднем")

1.8525820064544678 время работы на 1 записи в среднем


In [78]:
transcriptions[1]

{'text': ' Сейчас я тебе покажу секретную команду в Роблоксе. Чтобы её активировать поставь лайк и подпишись, а также введи в чат команду ILOVEYOU. Когда вы её введёте у вас на экране появится вот такой клоун. Скрипт был создан одним из создателей Роблокса и работает только в играх, где есть Роблос девелопер сервис. Чтобы убрать скример нужно всего',
 'chunks': [{'timestamp': (0.0, 2.56),
   'text': ' Сейчас я тебе покажу секретную команду в Роблоксе.'},
  {'timestamp': (2.56, 4.88),
   'text': ' Чтобы её активировать поставь лайк и подпишись, а также'},
  {'timestamp': (4.88, 6.8), 'text': ' введи в чат команду ILOVEYOU.'},
  {'timestamp': (6.8, 9.12),
   'text': ' Когда вы её введёте у вас на экране появится вот'},
  {'timestamp': (9.12, 10.46), 'text': ' такой клоун.'},
  {'timestamp': (10.46, 12.2),
   'text': ' Скрипт был создан одним из создателей Роблокса'},
  {'timestamp': (12.2, 14.6),
   'text': ' и работает только в играх, где есть Роблос девелопер'},
  {'timestamp': (14.6, 

In [79]:
transcriptions[1]

{'text': ' Сейчас я тебе покажу секретную команду в Роблоксе. Чтобы её активировать поставь лайк и подпишись, а также введи в чат команду ILOVEYOU. Когда вы её введёте у вас на экране появится вот такой клоун. Скрипт был создан одним из создателей Роблокса и работает только в играх, где есть Роблос девелопер сервис. Чтобы убрать скример нужно всего',
 'chunks': [{'timestamp': (0.0, 2.56),
   'text': ' Сейчас я тебе покажу секретную команду в Роблоксе.'},
  {'timestamp': (2.56, 4.88),
   'text': ' Чтобы её активировать поставь лайк и подпишись, а также'},
  {'timestamp': (4.88, 6.8), 'text': ' введи в чат команду ILOVEYOU.'},
  {'timestamp': (6.8, 9.12),
   'text': ' Когда вы её введёте у вас на экране появится вот'},
  {'timestamp': (9.12, 10.46), 'text': ' такой клоун.'},
  {'timestamp': (10.46, 12.2),
   'text': ' Скрипт был создан одним из создателей Роблокса'},
  {'timestamp': (12.2, 14.6),
   'text': ' и работает только в играх, где есть Роблос девелопер'},
  {'timestamp': (14.6, 

In [80]:
transcriptions[2]

{'text': ' Субтитры сделал DimaTorzok',
 'chunks': [{'timestamp': (0.0, 10.94),
   'text': ' Субтитры сделал DimaTorzok'}]}

In [81]:
transcriptions[3]

{'text': ' Что он о тебе думает прямо сейчас? Слушай, думает, что ты, знаешь, для него непосильная ноша. Как будто бы, да, королева мечей. Чувствует то, что ты по характеру гораздо сильнее, и то, что ему с тобой будет очень сложно. Но при этом есть желания, и желания не совсем приличные. Проявляться боится, и, честно говоря, сам проявляться не будет. Я бы на твоем месте вообще подумала, нужен ли тебе такой партнер, который слабее тебя. Но выводы делай сама.',
 'chunks': [{'timestamp': (0.0, 2.12),
   'text': ' Что он о тебе думает прямо сейчас?'},
  {'timestamp': (6.0, 10.22),
   'text': ' Слушай, думает, что ты, знаешь, для него непосильная ноша.'},
  {'timestamp': (10.88, 13.4), 'text': ' Как будто бы, да, королева мечей.'},
  {'timestamp': (14.04, 19.12),
   'text': ' Чувствует то, что ты по характеру гораздо сильнее, и то, что ему с тобой будет очень сложно.'},
  {'timestamp': (19.6, 23.8),
   'text': ' Но при этом есть желания, и желания не совсем приличные.'},
  {'timestamp': (24

| model          | GPU usage|
|whisper-large-v3| 4784MiB  |

#### Пробуем ускорить батчами

In [54]:
import librosa

In [60]:
# Функция для загрузки аудио
def load_audio(audio_paths):
    audios = []
    for path in audio_paths:
        audio, sr = librosa.load(path, sr=16000)
        audios.append(audio)
    return audios

# Функция для пакетной обработки аудио
def transcribe_batch(audio_paths):
    audios = load_audio(audio_paths)
    inputs = processor(audios, sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        transcriptions = pipe(inputs["input_features"].to(torch_dtype))
    return transcriptions

In [57]:
len(audio_paths)

100

In [58]:
batch_size

16

In [61]:
# Параллельная обработка аудиофайлов батчами
transcriptions = []
start_time = time.time()

# Разбиваем список путей на батчи
for i in range(0, len(audio_paths), batch_size):
    batch_paths = audio_paths[i:i + batch_size]
    batch_transcriptions = transcribe_batch(batch_paths)
    transcriptions.extend(batch_transcriptions)

time_work = time.time() - start_time
print(f"Время выполнения: {time_work:.2f} секунд")

ValueError: We expect a numpy ndarray as input, got `<class 'torch.Tensor'>`

не помогло, идем дальше

### проба либы whisper

тратит 10 Гб ГПУ, батч поддержки нет. скорость таже 

пробовал использовать [тут](batch_transcriptions) батч по путям к mp3 результат - не работает 

In [23]:
import whisper
import time
import numpy as np

In [15]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:58<00:00, 53.0MiB/s]


In [16]:
batch_size_n = 2

In [19]:
# Параллельная обработка аудиофайлов батчами
transcriptions = []
start_time = time.time()

# Разбиваем список путей на батчи
for i in range(0, len(audio_paths), batch_size_n):
    batch_paths = audio_paths[i:i + batch_size_n]
    batch_transcriptions = model.transcribe(batch_paths)
    transcriptions.extend(batch_transcriptions)
    break
time_work = time.time() - start_time
print(f"Время выполнения: {time_work:.2f} секунд")

TypeError: expected np.ndarray (got list)

In [20]:
batch_paths

['../../data_audio/0_b0_e9_ef285e0241139fc611318ed33071_fhd.mp3',
 '../../data_audio/1_39_6c_b31bc6864bef9d8a96814f1822ca_fhd.mp3']

In [31]:
%%time
batch_transcriptions = model.transcribe(batch_paths[1])

CPU times: user 7.72 s, sys: 195 ms, total: 7.92 s
Wall time: 5.3 s


In [32]:
batch_transcriptions

{'text': ' Сейчас я тебе покажу секретную команду в Роблоксе. Чтобы её активировать поставь лайк и подпишись, а также введи в чат команду ILOVEYOU. Когда вы её введёте у вас на экране появится вот такой клоун. Скрипт был создан одним из создателей Роблокса и работает только в играх, где есть Роблос девелопер сервис. Чтобы убрать скример нужно всего то выйти из игры, но лучше не проверяя.',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 2.56,
   'text': ' Сейчас я тебе покажу секретную команду в Роблоксе.',
   'tokens': [50365,
    23590,
    2552,
    14656,
    7240,
    29294,
    22869,
    31242,
    9882,
    46180,
    9031,
    740,
    6325,
    2061,
    693,
    2637,
    28340,
    13,
    50493],
   'temperature': 0.0,
   'avg_logprob': -0.19574242953596444,
   'compression_ratio': 1.8756906077348066,
   'no_speech_prob': 0.005552722606807947},
  {'id': 1,
   'seek': 0,
   'start': 2.56,
   'end': 4.88,
   'text': ' Чтобы её активировать поставь лайк и под

In [34]:
%%time
batch_transcriptions = model.transcribe(audio_paths[2])

CPU times: user 3.86 s, sys: 564 ms, total: 4.42 s
Wall time: 1.78 s


In [35]:
batch_transcriptions['text']

" I wanna spend some time with her Cause I can't get her through Type of bitch, I don't know who Falling in love"

In [ ]:
transcriptions